<a href="https://colab.research.google.com/github/Nubiancodingdelight/ACS-Project-Repository-/blob/main/Matrix_Creation__Draft1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#mounts google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import re

In [ ]:
# sets correct path
%cd drive/MyDrive/'ACS Research'/'Beginning Data'

/content/drive/MyDrive/ACS Research/Beginning Data


In [ ]:
%ls

 bowtie2/                  NC_012967.1.fasta   seq_1.fastq.gz        SRR030257_1.fastq
 condacolab_install.log    Output/             seq_2.fastq.gz        SRR030257_1.fastq.gz
'Data Exploration.ipynb'   readme.txt          seqtk/                SRR030257_2.fastq
 hg19/                     README.txt          simulation-data.tgz
 MyDrive/                  sam2codfreq         SRR030257_1.fasta


In [ ]:
!head bowtie2/SRR030257.sam

@HD	VN:1.5	SO:unsorted	GO:query
@SQ	SN:NC_012967.1	LN:4629812
@PG	ID:bowtie2	PN:bowtie2	VN:2.5.2	CL:"/usr/local/bin/bowtie2-align-s --wrapper basic-0 -t -x bowtie2/NC_012967.1 -S bowtie2/SRR030257.sam -1 SRR030257_1.fastq -2 SRR030257_2.fastq"
SRR030257.1	99	NC_012967.1	950180	42	36M	=	950295	151	TTACACTCCTGTTAATCCATACAGCAACAGTATTGG	AAA;A;AA?A?AAAAA?;?A?1A;;????566)=*1	AS:i:-2	XN:i:0	XM:i:1	XO:i:0	XG:i:0	NM:i:1	MD:Z:32C3	YS:i:-5	YT:Z:CP
SRR030257.1	147	NC_012967.1	950295	42	36M	=	950180	-151	ACACTAAAGACAAAGAAATCACATTGACAAAGTTAA	</<+9*AA/A)AAAAAA+A9A>A9AAAAAAAAAAAA	AS:i:-5	XN:i:0	XM:i:2	XO:i:0	XG:i:0	NM:i:2	MD:Z:5G11G18	YS:i:-2	YT:Z:CP
SRR030257.2	99	NC_012967.1	2349583	42	36M	=	2349702	155	TAAGCCAGTCGCCATGGAATATCTGCTTTATTTAGC	AAAAAAAAAA;A?AA;A?AAAAAA8????+9=&=1;	AS:i:-2	XN:i:0	XM:i:1	XO:i:0	XG:i:0	NM:i:1	MD:Z:32A3	YS:i:0	YT:Z:CP
SRR030257.2	147	NC_012967.1	2349702	42	36M	=	2349583	-155	CGCTGCGCAGTACCAGTGTGTTGGCGAGGTACGCAG	<?9>?>AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA	AS:i:0	XN:i:0	XM:i:0	XO:i:0

In [ ]:
#parse into data frame (Excludes last 2 columns)
sam_frame = pd.read_csv("bowtie2/SRR030257.sam", sep='\t', skiprows =3, usecols=[0, 1,2,3,4,5,6,7,8,9,10],
                       names=["Query Name", "Flag", "Reference Name", "Position", "Mapping Quality", 'CIGAR',
                              "Reference Name for Mate", "Position of Mate","Template Length", "Sequence", "Quality String"], header=None)

In [ ]:
sam_frame.head()

,Query Name,Flag,Reference Name,Position,Mapping Quality,CIGAR,Reference Name for Mate,Position of Mate,Template Length,Sequence,Quality String
0,SRR030257.1,99,NC_012967.1,950180,42,36M,=,950295,151,TTACACTCCTGTTAATCCATACAGCAACAGTATTGG,AAA;A;AA?A?AAAAA?;?A?1A;;????566)=*1
1,SRR030257.1,147,NC_012967.1,950295,42,36M,=,950180,-151,ACACTAAAGACAAAGAAATCACATTGACAAAGTTAA,</<+9*AA/A)AAAAAA+A9A>A9AAAAAAAAAAAA
2,SRR030257.2,99,NC_012967.1,2349583,42,36M,=,2349702,155,TAAGCCAGTCGCCATGGAATATCTGCTTTATTTAGC,AAAAAAAAAA;A?AA;A?AAAAAA8????+9=&=1;
3,SRR030257.2,147,NC_012967.1,2349702,42,36M,=,2349583,-155,CGCTGCGCAGTACCAGTGTGTTGGCGAGGTACGCAG,<?9>?>AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
4,SRR030257.3,99,NC_012967.1,3354980,1,36M,=,3355102,158,TTGGTAGGCCTGAGTGGACTTGAACCACCGACCTCA,AAAAAAAAAAAAAAA>AAAAA>>AA?<???-><><;


In [ ]:
#create data frame
matrix_frame= pd.DataFrame(columns=['A','T','C','G'])
matrix_frame.head()


,A,T,C,G


In [ ]:
len(sam_frame[sam_frame['CIGAR'] == '*'])

371144

In [ ]:
cigar_list = re.split(r'(\d+)', '2M242D4M3D3M')
print(cigar_list)

['', '2', 'M', '242', 'D', '4', 'M', '3', 'D', '3', 'M']


In [ ]:
#create datafcrame
for ind in range(len(sam_frame[sam_frame['CIGAR'] != '*'])):
  cigar_list = re.split(r'(\d+)', sam_frame['CIGAR'][ind])[1::]
  seq_list = list(sam_frame['Sequence'][ind])
  #checks to see that there is only one type of value for the CIGAR
  if len(cigar_list) ==2:
    #if it is a matched a sequence of 1's based on the base are added
    if cigar_list[1] == 'M':
      for item in seq_list:
        bases = ['A','T','C','G']
        #if statement causes all values not in the base list to be ignored.
        if item in bases:
          bases.remove(item)
          df_var= {item:1, bases.pop():0, bases.pop():0,bases.pop():0}
          matrix_frame= matrix_frame.append(df_var, ignore_index=True)
    else:
      matrix_frame = matrix_frame.append(pd.Series(0, index=matrix_frame.columns), ignore_index=True)
  else:
    for time in range(int(len(cigar_list)/2)):
      if cigar_list[(2*time)+1] == 'M':
        for item in seq_list:
          bases = ['A','T','C','G']
          #if statement causes all values not in the base list to be ignored.
          if item in bases:
            bases.remove(item)
            df_var= {item:1, bases.pop():0, bases.pop():0,bases.pop():0}
            matrix_frame= matrix_frame.append(df_var, ignore_index=True)
    else:
      matrix_frame = matrix_frame.append(pd.Series(0, index=matrix_frame.columns), ignore_index=True)





Streaming output truncated to the last 5000 lines.
<ipython-input-67-16f18bbcc7f3>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matrix_frame= matrix_frame.append(df_var, ignore_index=True)
<ipython-input-67-16f18bbcc7f3>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matrix_frame= matrix_frame.append(df_var, ignore_index=True)
<ipython-input-67-16f18bbcc7f3>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matrix_frame= matrix_frame.append(df_var, ignore_index=True)
<ipython-input-67-16f18bbcc7f3>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matrix_frame= matrix_frame.append(df_var, ignore_index=True)
<ipython-input-67-16f

In [ ]:
matrix_frame.head()

,A,T,C,G
0,0,1,0,0
1,0,1,0,0
2,1,0,0,0
3,0,0,1,0
4,1,0,0,0


from matplotlib import pyplot as plt
import seaborn as sns
_df_0.groupby('A').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_1.groupby('T').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_2.groupby('C').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  from matplotlib import pyplot as plt
  import seaborn as sns
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['T']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'T'}, axis=1)
              .sort_values('T', ascending=True))
  xs = counted['T']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_3.sort_values('T', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('A')):
  _plot_series(series, series_name, i)
  fig.legend(title='A', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('T')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  from matplotlib import pyplot as plt
  import seaborn as sns
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['T']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'T'}, axis=1)
              .sort_values('T', ascending=True))
  xs = counted['T']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_4.sort_values('T', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('T')):
  _plot_series(series, series_name, i)
  fig.legend(title='T', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('T')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  from matplotlib import pyplot as plt
  import seaborn as sns
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['T']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'T'}, axis=1)
              .sort_values('T', ascending=True))
  xs = counted['T']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_5.sort_values('T', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('C')):
  _plot_series(series, series_name, i)
  fig.legend(title='C', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('T')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['T'].value_counts()
    for x_label, grp in _df_6.groupby('A')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('A')
_ = plt.ylabel('T')

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['C'].value_counts()
    for x_label, grp in _df_7.groupby('T')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('T')
_ = plt.ylabel('C')